In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

from models import BachNet
from utils import preprocessing_utils

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
bach = BachNet.BachNet()

In [4]:
train_path = "data/train/input/"
path_to_original_images = "original/"
path_to_gt_images = "gt/"

gt_tag = "gt"

receptive_field_shape = (23, 23)

In [5]:
print("Loading Images")

X = []
Y = []

files = os.listdir(train_path)

for i, filename in enumerate(files):
    
    sys.stdout.write("\rProgress: %.2f%% || %d/%d" % (i / len(files),
                                                      i,
                                                      len(files)))
    
    
    f_name, ext = os.path.splitext(filename)
    
    if gt_tag in f_name.split("_"):
        continue
        
    gt_filename = f_name + "_" + gt_tag
    
    gt_path = os.path.join(train_path, (gt_filename + ext))
        
    if not os.path.isfile(gt_path):
        continue
        
    foldername = "data/train/chopin/" + f_name
        
    image_path = os.path.join(train_path, filename)
        
    img = cv2.imread(image_path, 0)
    
    x = preprocessing_utils.prepare_input_images(img, width=receptive_field_shape[0],
                                                 height=receptive_field_shape[1])
    
    X.append(x)

    gt = cv2.imread(gt_path, 0)
    y = np.zeros_like(gt)
    y[gt == 255] = 1
    y = y.flatten()
    
    Y.append(y)
    
X = np.concatenate(X)
Y = np.concatenate(Y)

sys.stdout.write("\rProgress: Done! || %d/%d" % (len(files),
                                                len(files)))
sys.stdout.flush()

Loading Images
Progress: Done! || 319/319

In [6]:
print(X.shape, Y.shape)

(1450000, 23, 23, 1) (1450000,)


In [7]:
bach.build(receptive_field_shape[0], receptive_field_shape[1], 1)

In [8]:
bach.model.fit(X, Y)

Epoch 1/10
1450000/1450000 [==============================] - 1028s - loss: 0.1932 - acc: 0.9526  
Epoch 2/10
  45120/1450000 [..............................] - ETA: 997s - loss: 0.1917 - acc: 0.9531

KeyboardInterrupt: 

In [9]:
bach.model.save("models/saved_model/Bach/model.h5")